In [2]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd

In [2]:
yf.pdr_override()
pdr.get_data_yahoo("AMZN", start="2020-03-10", end="2024-04-18").to_csv("amazon_stock.csv")

[*********************100%%**********************]  1 of 1 completed


In [3]:
layoffs = pd.read_csv(r"layoffs/layoffs.csv")
layoffs

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,The Good Glamm Group,New Delhi,Retail,150.0,0.15,2024-04-18,Unknown,India,180.0
1,Stability AI,London,AI,20.0,NaN,2024-04-18,Unknown,United Kingdom,174.0
2,ConnectWise,Tampa Bay,Other,NaN,NaN,2024-04-17,Acquired,United States,1.0
3,Google,SF Bay Area,Consumer,NaN,NaN,2024-04-17,Post-IPO,United States,26.0
4,Rivian,Detroit,Transportation,NaN,0.01,2024-04-17,Post-IPO,United States,10700.0
...,...,...,...,...,...,...,...,...,...
3563,Service,Los Angeles,Travel,NaN,1.00,2020-03-16,Seed,United States,5.1
3564,HopSkipDrive,Los Angeles,Transportation,8.0,0.10,2020-03-13,Unknown,United States,45.0
3565,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0
3566,Tamara Mellon,Los Angeles,Retail,20.0,0.40,2020-03-12,Series C,United States,90.0


In [4]:
symbols = pd.read_csv(r"nasdaq_screener.csv")
symbols

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$139.23,1.7100,1.243%,4.080209e+10,United States,1999.0,701427,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$36.575,0.6350,1.767%,6.567370e+09,United States,2016.0,2937634,Industrials,Aluminum
2,AACG,ATA Creativity Global American Depositary Shares,$0.92,0.0192,2.131%,2.944012e+07,China,2008.0,11093,Real Estate,Other Consumer Services
3,AACI,Armada Acquisition Corp. I Common Stock,$11.74,0.1200,1.033%,0.000000e+00,United States,2021.0,9738,Finance,Blank Checks
4,AACIW,Armada Acquisition Corp. I Warrant,$0.14,0.0000,0.00%,0.000000e+00,United States,2021.0,10271,Finance,Blank Checks
...,...,...,...,...,...,...,...,...,...,...,...
7130,ZVRA,Zevra Therapeutics Inc. Common Stock,$4.79,0.0400,0.842%,2.080114e+08,United States,NaN,132607,Health Care,Biotechnology: Pharmaceutical Preparations
7131,ZVSA,ZyVersa Therapeutics Inc. Common Stock,$5.54,-0.2000,-3.484%,4.207552e+06,United States,2022.0,368385,Health Care,Biotechnology: Pharmaceutical Preparations
7132,ZWS,Zurn Elkay Water Solutions Corporation Common ...,$31.74,0.1100,0.348%,5.481398e+09,United States,2012.0,303610,Industrials,Industrial Machinery/Components
7133,ZYME,Zymeworks Inc. Common Stock,$9.165,0.1650,1.833%,6.467578e+08,United States,NaN,361393,Health Care,Biotechnology: Pharmaceutical Preparations


In [7]:
# attempt to map stock symbols on dataframe
layoffs["join"] = 1
symbols["join"] = 1
df_full = layoffs.merge(symbols, on="join").drop("join", axis=1)

symbols.drop("join", axis=1, inplace=True)

df_full['match'] = df_full.apply(
    lambda x: x["Name"].find(x['company']), axis=1).ge(0)

print(df_full[df_full['match']])

           company       location        industry  total_laid_off  \
34023       Rivian        Detroit  Transportation             NaN   
42154     Take-Two  New York City        Consumer           579.0   
56404        Tesla         Austin  Transportation         14000.0   
58747       Criteo          Paris       Marketing           140.0   
81250       Fabric  New York City       Logistics            30.0   
...            ...            ...             ...             ...   
25428489   Service    Los Angeles          Travel             NaN   
25428562   Service    Los Angeles          Travel             NaN   
25428596   Service    Los Angeles          Travel             NaN   
25428973   Service    Los Angeles          Travel             NaN   
25429105   Service    Los Angeles          Travel             NaN   

          percentage_laid_off        date     stage        country  \
34023                    0.01  2024-04-17  Post-IPO  United States   
42154                    0.05  

In [8]:
df_full[df_full['match']].to_csv("layoffs_and_stock_info.csv")

In [11]:
layoffs_and_stock_df = pd.read_csv("layoffs_and_stock_info.csv")

In [20]:
# clean up
layoffs_and_stock_df.drop(columns="Unnamed: 0").loc[layoffs_and_stock_df["Country"]==layoffs_and_stock_df["country"]].drop_duplicates(subset="Name").to_csv('layoffs_and_stock_info_CLEANED.csv')

Pull stock data for companies

In [23]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import os

In [24]:
start_date = "2020-03-10"
end_date = "2024-04-18"
layoffs_and_stock_df_clean = pd.read_csv("layoffs_and_stock_info_CLEANED.csv")

In [25]:
yf.pdr_override()
for i, row in layoffs_and_stock_df_clean.iterrows():
    symbol = row["Symbol"]
    industry = row["industry"]

    directory = "stocks\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    pdr.get_data_yahoo(symbol, start=start_date, end=end_date).to_csv(directory + f"\\{symbol}.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Make layoffs datasets

In [26]:
import pandas as pd
import os

In [ ]:
start_date = "2020-03-10"
end_date = "2024-04-18"
layoffs_and_stock_df_clean = pd.read_csv("layoffs_and_stock_info_CLEANED.csv")

In [29]:
layoffs_and_stock_df_clean

,Unnamed: 0,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised,...,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,1568,1stdibs,New York City,Retail,70.0,0.17,4/2/2020,Series D,United States,253.0,...,1stdibs.com Inc. Common Stock,$5.74,-0.1000,-1.71%,2.281410e+08,United States,2021.0,33618,Consumer Discretionary,Catalog/Specialty Distribution
1,247,23andMe,SF Bay Area,Healthcare,71.0,0.11,8/8/2023,Post-IPO,United States,1100.0,...,23andMe Holding Co. Class A Common Stock,$0.50,-0.0056,-1.11%,2.409754e+08,United States,NaN,1757192,Health Care,Biotechnology: Pharmaceutical Preparations
2,66,2U,Washington D.C.,Education,NaN,NaN,1/22/2024,Post-IPO,United States,426.0,...,2U Inc. Common Stock,$0.33,0.0594,22.12%,2.742688e+07,United States,2014.0,2778479,Technology,Computer Software: Prepackaged Software
3,675,8x8,SF Bay Area,Support,155.0,0.07,1/18/2023,Post-IPO,United States,253.0,...,8x8 Inc Common Stock,$2.23,-0.0200,-0.89%,2.752298e+08,United States,NaN,685314,Technology,EDP Services
4,208,Absci,Portland,Healthcare,30.0,0.15,9/5/2023,Post-IPO,United States,238.0,...,Absci Corporation Common Stock,$5.10,0.1150,2.31%,5.746374e+08,United States,2021.0,446921,Health Care,Biotechnology: Commercial Physical & Biologica...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,365,ZipRecruiter,Los Angeles,Recruiting,270.0,0.20,5/31/2023,Post-IPO,United States,769.0,...,ZipRecruiter Inc. Class A Common Stock,$10.24,-0.0600,-0.58%,1.014513e+09,United States,2021.0,533814,Technology,Computer Software: Programming Data Processing
364,45,Zoom,SF Bay Area,Other,150.0,0.02,2/1/2024,Post-IPO,United States,276.0,...,Zoom Video Communications Inc. Class A Common ...,$61.43,-1.4000,-2.23%,1.889339e+10,United States,2019.0,2065385,Technology,Computer Software: Programming Data Processing
365,361,ZoomInfo,Vancouver,Sales,120.0,0.03,6/2/2023,Post-IPO,United States,7.0,...,ZoomInfo Technologies Inc Common Stock,$16.19,-0.2900,-1.76%,6.147383e+09,United States,2020.0,5833564,Technology,Computer Software: Prepackaged Software
366,531,Zscaler,SF Bay Area,Security,177.0,0.03,3/2/2023,Post-IPO,United States,148.0,...,Zscaler Inc. Common Stock,$178.35,1.9800,1.12%,2.672717e+10,United States,2018.0,1886844,Technology,EDP Services


In [35]:
df = layoffs_and_stock_df_clean[["date","total_laid_off","percentage_laid_off"]].groupby("date").agg({"total_laid_off":"sum","percentage_laid_off":"mean"})

In [37]:
for industry in layoffs_and_stock_df_clean['industry']:

    directory = "layoffs\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    df = layoffs_and_stock_df_clean.loc[layoffs_and_stock_df_clean['industry'] == industry]

    df[["date","total_laid_off","percentage_laid_off"]].groupby("date").agg({"total_laid_off":"sum","percentage_laid_off":"mean"}).to_csv(directory + f"\\layoffs.csv")